# 🎙️ Entrenamiento de Voz Piper TTS en Google Colab

Este notebook permite entrenar modelos de voz Piper TTS con GPU gratuita de Google Colab.

**GPU recomendada:** T4 o superior  
**Tiempo por época:** ~10-30 minutos con GPU (vs 30-60 min en CPU)

---

## 1️⃣ Configuración Inicial

Verificar GPU y montar Google Drive

In [1]:
# Verificar GPU disponible
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Wed Jan  7 11:30:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Crear directorio de trabajo
!mkdir -p /content/piper-training
%cd /content/piper-training

Mounted at /content/drive
/content/piper-training


## 2️⃣ Instalación de Dependencias

Instalar Piper y todas las dependencias

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [4]:
!conda create -n piper_env python=3.10

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/piper_env

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2026.1.4   |       hbd8a1cb_0         143 KB  conda-forge
    icu-78.1                   |       h33c6efd_0        12.1 MB  conda-forge
    ld_impl_linux-64-2.45      |default_hbd61a6d_105         714 KB  conda-forge
    libexpat-2.7.3             |       hecca717_0          75 KB  conda-forge
    libffi-3.5.2               |       h9ec8514_0          56 KB  conda-forg

In [1]:
%%bash
set -e
echo "📦 Instalando dependencias del sistema..."
apt-get update -qq
apt-get install -y -qq espeak-ng wget git 2>&1 | grep -v "debconf"
echo "✅ espeak-ng instalado"
espeak-ng --version

📦 Instalando dependencias del sistema...
Selecting previously unselected package libpcaudio0:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libpcaudio0_1.1-6build2_amd64.deb ...
Unpacking libpcaudio0:amd64 (1.1-6build2) ...
Selecting previously unselected package libsonic0:amd64.
Preparing to unpack .../libsonic0_0.2.0-11build1_amd64.deb ...
Unpacking libsonic0:amd64 (0.2.0-11build1) ...
Selecting previously unselected package espeak-ng-data:amd64.
Preparing to unpack .../espeak-ng-data_1.50+dfsg-10ubuntu0.1_amd64.deb ...
Unpacking espeak-ng-data:amd64 (1.50+dfsg-10ubuntu0.1) ...
Selecting previously unselected package libespeak-ng1:amd64.
Preparing to unpack .../libespeak-ng1_1.50+dfsg-10ubuntu0.1_amd64.deb ...
Unpacking libespeak-ng1:amd64 (1.50+dfsg-10ubuntu0.1) ...
Selecting previously unselected package espeak-ng.
Preparing to unpack .../espeak-ng_1.50+dfsg-10ubuntu0.1_amd64.deb ...
Unpacking espeak-ng (1.50+dfsg-10ubuntu0.1

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
%%bash
echo "📦 Instalando piper-phonemize..."
cd /content/piper-training

if [ ! -d "piper_phonemize" ]; then
    echo "Descargando piper-phonemize..."
    wget --show-progress \
        https://github.com/rhasspy/piper-phonemize/releases/download/2023.11.14-4/piper-phonemize_linux_x86_64.tar.gz \
        || { echo "❌ Error descargando piper-phonemize"; exit 1; }

    echo "Descomprimiendo..."
    tar -xzf piper-phonemize_linux_x86_64.tar.gz \
        || { echo "❌ Error descomprimiendo piper-phonemize"; exit 1; }

    rm piper-phonemize_linux_x86_64.tar.gz
    echo "✅ piper-phonemize instalado"

    # Verificar instalación
    if [ -d "piper_phonemize" ]; then
        echo "✅ Directorio piper_phonemize creado correctamente"
        ls -la piper_phonemize/ | head -10
    fi
else
    echo "✅ piper-phonemize ya existe"
    ls -la piper_phonemize/ | head -10
fi

📦 Instalando piper-phonemize...
Descargando piper-phonemize...
Descomprimiendo...
✅ piper-phonemize instalado
✅ Directorio piper_phonemize creado correctamente
total 24
drwxr-xr-x 6 root root 4096 Nov 14  2023 .
drwxr-xr-x 3 root root 4096 Jan  7 11:31 ..
drwxr-xr-x 2 root root 4096 Nov 14  2023 bin
drwxr-xr-x 5 root root 4096 Nov 14  2023 include
drwxr-xr-x 3 root root 4096 Nov 14  2023 lib
drwxr-xr-x 4 root root 4096 Nov 14  2023 share


--2026-01-07 11:31:52--  https://github.com/rhasspy/piper-phonemize/releases/download/2023.11.14-4/piper-phonemize_linux_x86_64.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/650341719/5cf1ab84-f05c-4c1b-90f5-cff3a87a525b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-07T12%3A14%3A22Z&rscd=attachment%3B+filename%3Dpiper-phonemize_linux_x86_64.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-07T11%3A13%3A35Z&ske=2026-01-07T12%3A14%3A22Z&sks=b&skv=2018-11-09&sig=Trv6x6xZHdUI0nC0f1dGZyIlTUVrYglsPSTmCnvaM7Q%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2Nzc4NzMxMiwibmJmIjoxNzY3Nzg1

In [3]:
%%bash
set -e
echo "📦 Clonando Piper..."
if [ ! -d "piper" ]; then
    git clone -q https://github.com/rhasspy/piper.git
    echo "✅ Piper clonado"
else
    echo "✅ Piper ya existe"
fi

📦 Clonando Piper...
✅ Piper clonado


In [4]:
# @title 📦 INSTALACIÓN COMPLETA + VERIFICACIÓN
# Esta celda instala TODO y verifica en un solo paso

import os
import subprocess
import sys

print("📦 CONFIGURANDO ENTORNO PARA PIPER-TRAINING")
print("=" * 50)

# Crear directorio de trabajo
!mkdir -p /content/piper-training
%cd /content/piper-training

# Clonar Piper si no existe
if not os.path.exists("/content/piper-training/piper"):
    print("📥 Clonando repositorio piper...")
    !git clone -q https://github.com/rhasspy/piper.git
    print("   ✅ Piper clonado")
else:
    print("✅ Piper ya existe")

piper_python_dir = "/content/piper-training/piper/src/python"

if not os.path.exists(piper_python_dir):
    print("❌ ERROR: piper/src/python no existe")
    raise SystemExit()

# 1. Degradar pip
print("🔧 Ajustando pip...")
!pip install --quiet "pip<24.1"

# 2. Instalar TODAS las dependencias
# Forzamos versiones específicas para evitar conflictos
!pip install --quiet "pytorch-lightning==1.7.7" "torchmetrics==0.11.4"
!pip install --quiet piper-phonemize==1.1.0
!pip install --quiet librosa
!pip install --quiet "numpy<2.0.0" "scipy==1.11.4"
!pip install --quiet git+https://github.com/resemble-ai/monotonic_align.git

# 3. Copiar monotonic_align
print("🔧 Copiando monotonic_align...")
!cp -r /usr/local/lib/python3.11/site-packages/monotonic_align {piper_python_dir}/

# 4. Parchear piper_train (__init__.py)
print("🔧 Parcheando monotonic_align...")
init_file = f"{piper_python_dir}/piper_train/vits/monotonic_align/__init__.py"
new_init = '''"""Monotonic alignment search"""
import os, sys
_piper_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))))
import torch
import numpy as np
if _piper_dir not in sys.path:
    sys.path.append(_piper_dir)
from monotonic_align.core import maximum_path_c
def maximum_path(neg_cent, mask):
    device, dtype = neg_cent.device, neg_cent.dtype
    neg_cent = neg_cent.data.cpu().numpy().astype(np.float32)
    path = np.zeros(neg_cent.shape, dtype=np.int32)
    t_t_max = mask.sum(1)[:, 0].data.cpu().numpy().astype(np.int32)
    t_s_max = mask.sum(2)[:, 0].data.cpu().numpy().astype(np.int32)
    maximum_path_c(path, neg_cent, t_t_max, t_s_max)
    return torch.from_numpy(path).to(device=device, dtype=dtype)
'''
with open(init_file, 'w') as f:
    f.write(new_init)

# 5. PARCHE CRÍTICO PARA PYTORCH 2.6+ (weights_only=False)
print("🔧 Aplicando parche de seguridad para PyTorch...")
patch_file = f"{piper_python_dir}/piper_train/torch_patch.py"
patch_content = '''
import torch
import functools
# Monkey-patch torch.load to default weights_only=False for old checkpoints
_original_load = torch.load
@functools.wraps(_original_load)
def safe_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_load(*args, **kwargs)
torch.load = safe_load

# ------------------------------------------------------------------
# PATCH 2: PyTorch Lightning 1.7 vs PyTorch 2.x LR Scheduler Fix
# ------------------------------------------------------------------
try:
    import pytorch_lightning.core.optimizer
    # Disable the strict API validation check that fails with newer PyTorch
    def no_op(*args, **kwargs):
        pass
    pytorch_lightning.core.optimizer._validate_scheduler_api = no_op
except ImportError:
    pass
'''
with open(patch_file, 'w') as f:
    f.write(patch_content)

# Modificar __main__.py para importar el parche al inicio
main_file = f"{piper_python_dir}/piper_train/__main__.py"
with open(main_file, 'r') as f:
    content = f.read()
if 'torch_patch' not in content:
    new_content = "from . import torch_patch\n" + content
    with open(main_file, 'w') as f:
        f.write(new_content)

# 6. Instalar piper-train
print("🛠️ Instalando piper-train...")
%cd {piper_python_dir}
!pip install --quiet --no-deps -e .
%cd /content/piper-training

# 7. Configurar entorno
os.environ['LD_LIBRARY_PATH'] = f"/content/piper-training/piper_phonemize/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"

print("\n" + "=" * 50)
print("✅ INSTALACIÓN COMPLETADA")
print("=" * 50)

# 8. VERIFICACIÓN
print("\n🔍 VERIFICANDO ENTORNO...")
verify_code = '''
import sys
sys.path.insert(0, "/content/piper-training/piper/src/python")

import torch
print(f"PyTorch: {torch.__version__}")

import numpy as np
import scipy
print(f"numpy: {np.__version__}")
print(f"scipy: {scipy.__version__}")

import pytorch_lightning as pl
print(f"pytorch-lightning: {pl.__version__}")

import piper_train
print("piper_train: OK")
from piper_train.vits.monotonic_align import maximum_path
print("monotonic_align: OK")
import librosa
print(f"librosa: {librosa.__version__}")

# Verificar parche de torch
from piper_train import torch_patch
print("Torch patch: OK")

# Test
test_neg = torch.randn(1, 10, 20)
test_mask = torch.ones(1, 10, 20)
result = maximum_path(test_neg, test_mask)
print(f"Test maximum_path: {result.shape}")
print("\\n✅ TODO LISTO PARA ENTRENAR")
'''

result = subprocess.run(['python', '-c', verify_code], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print("❌ ERRORES:")
    print(result.stderr)

📦 CONFIGURANDO ENTORNO PARA PIPER-TRAINING
/content/piper-training
📥 Clonando repositorio piper...
   ✅ Piper clonado
🔧 Ajustando pip...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 151.9 M

In [ ]:
# Esta celda ya no es necesaria - todo está en la celda anterior
pass

## 3️⃣ Descargar Modelo Base

In [5]:
%%bash
set -e
mkdir -p models_base && cd models_base
if [ ! -f "en_US-lessac-high.ckpt" ]; then
    echo "📥 Descargando checkpoint (952 MB)..."
    wget -q --show-progress \
        "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/7bf647cb000d8c8319c6cdd4289dd6b7d0d3eeb8/en/en_US/lessac/high/epoch=2218-step=838782.ckpt" \
        -O en_US-lessac-high.ckpt
    echo "✅ Checkpoint descargado"
else
    echo "✅ Checkpoint ya existe"
fi

📥 Descargando checkpoint (952 MB)...
✅ Checkpoint descargado



     0K .......... .......... .......... .......... ..........  0% 4.96M 3m12s
    50K .......... .......... .......... .......... ..........  0% 32.2M 1m51s
   100K .......... .......... .......... .......... ..........  0% 11.4M 1m42s
   150K .......... .......... .......... .......... ..........  0% 16.3M 91s
   200K .......... .......... .......... .......... ..........  0% 17.6M 83s
   250K .......... .......... .......... .......... ..........  0% 31.7M 75s
   300K .......... .......... .......... .......... ..........  0% 12.1M 75s
   350K .......... .......... .......... .......... ..........  0%  277M 66s
   400K .......... .......... .......... .......... ..........  0% 42.4M 61s
   450K .......... .......... .......... .......... ..........  0% 63.5M 57s
   500K .......... .......... .......... .......... ..........  0% 45.0M 53s
   550K .......... .......... .......... .......... ..........  0% 18.0M 53s
   600K .......... .......... .......... .......... ..........  0%  3

## 4️⃣ Configurar Dataset

Sube tu dataset preprocesado o procesa uno nuevo

In [6]:
# OPCIÓN A: Copiar desde Google Drive
DRIVE_PATH = "/content/drive/MyDrive/sig"  # ⬅️ AJUSTAR

!mkdir -p datasets
!cp -r "{DRIVE_PATH}" datasets/
import os
DATASET_NAME = os.path.basename(DRIVE_PATH)
DATASET_DIR = f"datasets/{DATASET_NAME}"
print(f"✅ Dataset copiado: {DATASET_DIR}")

✅ Dataset copiado: datasets/sig


In [ ]:
# OPCIÓN B: Subir ZIP y preprocesar
from google.colab import files
uploaded = files.upload()  # Sube tu dataset.zip aquí

In [7]:
# @title 🧹 LIMPIEZA DE DATASET (Audios cortos)
# Filtra audios menores a 0.5s que causan error en VITS
import json
import os
from pathlib import Path
import librosa

DATASET_DIR = Path("datasets/sig")
JSONL_PATH = DATASET_DIR / "dataset.jsonl"
JSONL_FILTERED = DATASET_DIR / "dataset_filtered.jsonl"

print(f"🔍 Analizando {JSONL_PATH}...")

valid_lines = []
rejected = 0
MIN_DURATION = 1.0  # Segundos mínimos (Aumentado para seguridad)

if not JSONL_PATH.exists():
    print("❌ ERROR: No se encuentra dataset.jsonl")
else:
    with open(JSONL_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            # Usar campo audio_file que contiene la ruta relativa (ej: wavs/file.wav)
            audio_rel = item.get('audio_file')
            if not audio_rel:
                # Fallback por si acaso
                audio_id = item.get('id')
                wav_path = DATASET_DIR / "wavs" / f"{audio_id}.wav"
            else:
                wav_path = DATASET_DIR / audio_rel

            if wav_path.exists():
                try:
                    duration = librosa.get_duration(path=str(wav_path))
                    if duration >= MIN_DURATION:
                        valid_lines.append(line)
                    else:
                        print(f"⚠️ Ignorando (muy corto): {wav_path.name} ({duration:.2f}s)")
                        rejected += 1
                except Exception as e:
                    print(f"⚠️ Ignorando (corrupto/error): {wav_path.name} - {e}")
                    rejected += 1
            else:
                # IMPORTANTE: Si no existe el audio, NO lo añadimos
                print(f"⚠️ Ignorando (no encontrado): {wav_path}")
                rejected += 1

    print(f"\n✅ Generando dataset filtrado con {len(valid_lines)} muestras.")
    print(f"🗑️ Rechazados: {rejected}")

    with open(JSONL_FILTERED, 'w', encoding='utf-8') as f:
        f.writelines(valid_lines)

    # Reemplazar dataset original (backup primero)
    if len(valid_lines) == 0:
        print("❌ ERROR: dataset vacío. No se reemplaza nada.")
    elif rejected > 0:
        print(f"🔄 Reemplazando dataset.jsonl original (con {len(valid_lines)} audios)...")
        # Prevenir error si backup ya existe (windows/linux diff)
        if os.path.exists(DATASET_DIR / "dataset_backup.jsonl"):
            os.remove(DATASET_DIR / "dataset_backup.jsonl")
        os.rename(JSONL_PATH, DATASET_DIR / "dataset_backup.jsonl")
        os.rename(JSONL_FILTERED, JSONL_PATH)

# --- FIN LIMPIEZA ---

# Configuración de entrenamiento
DATASET_DIR = "datasets/sig"  # ⬅️ AJUSTAR
MAX_EPOCHS = 3000  # Aumentado porque el checkpoint base ya tiene >2000 épocas
BATCH_SIZE = 8  # Reducido para estabilidad en VRAM T4
CHECKPOINT_EPOCHS = 5
VALIDATION_SPLIT = 0.05
NUM_TEST_EXAMPLES = 0
PRECISION = "32"
QUALITY = "high"  # 🔥 CRÍTICO: Debe coincidir con el checkpoint

# Entrenar modelo con piper_train
!python -m piper_train \
  --dataset-dir {DATASET_DIR} \
  --accelerator gpu \
  --devices 1 \
  --batch-size {BATCH_SIZE} \
  --validation-split {VALIDATION_SPLIT} \
  --num-test-examples {NUM_TEST_EXAMPLES} \
  --max_epochs {MAX_EPOCHS} \
  --resume_from_checkpoint models_base/en_US-lessac-high.ckpt \
  --checkpoint-epochs {CHECKPOINT_EPOCHS} \
  --precision {PRECISION} \
  --quality {QUALITY}  # ⬅️ IMPORTANTE


🔍 Analizando datasets/sig/dataset.jsonl...

✅ Generando dataset filtrado con 700 muestras.
🗑️ Rechazados: 0
DEBUG:piper_train:Namespace(dataset_dir='datasets/sig', checkpoint_epochs=5, quality='high', resume_from_single_speaker_checkpoint=None, logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices='1', gpus=None, auto_select_gpus=False, tpu_cores=None, ipus=None, enable_progress_bar=True, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=None, max_epochs=3000, min_epochs=None, max_steps=-1, min_steps=None, max_time=None, limit_train_batches=None, limit_val_batches=None, limit_test_batches=None, limit_predict_batches=None, val_check_interval=None, log_every_n_steps=50, accelerator='gpu', strategy=None, sync_batchnorm=False, precision=32, enable_model_summary=True, weights_save_path=None, num_sanity_val_steps=2, resume_from_

### 📋 Verificar Dataset

Antes de entrenar, verifica la estructura del dataset

In [ ]:
# Verificar estructura del dataset
import json
from pathlib import Path

DATASET_DIR = "datasets/sig"  # ⬅️ AJUSTAR si es necesario
dataset_path = Path(DATASET_DIR)

print("🔍 Verificando dataset...\n")

# Verificar archivos requeridos
required_files = ['config.json', 'dataset.jsonl']
for file in required_files:
    file_path = dataset_path / file
    if file_path.exists():
        print(f"✅ {file}")
        if file == 'config.json':
            with open(file_path, 'r') as f:
                config = json.load(f)
                print(f"   - Audio: {config.get('audio', {}).get('sample_rate')} Hz")
                print(f"   - Idioma: {config.get('espeak', {}).get('voice')}")
    else:
        print(f"❌ {file} no encontrado")

# Contar muestras
jsonl_path = dataset_path / 'dataset.jsonl'
if jsonl_path.exists():
    with open(jsonl_path, 'r') as f:
        num_samples = sum(1 for _ in f)
    print(f"\n📊 Total de muestras: {num_samples}")

    # Calcular duración aproximada de entrenamiento
    samples_per_epoch = num_samples
    time_per_sample = 0.3  # segundos aproximados por muestra en GPU T4
    time_per_epoch = (samples_per_epoch * time_per_sample) / 60
    print(f"⏱️ Tiempo estimado por época: ~{time_per_epoch:.1f} minutos (GPU T4)")
else:
    print("❌ No se pudo contar muestras")

🔍 Verificando dataset...

✅ config.json
   - Audio: 22050 Hz
   - Idioma: es
✅ dataset.jsonl

📊 Total de muestras: 700
⏱️ Tiempo estimado por época: ~3.5 minutos (GPU T4)


## 5️⃣ Entrenar Modelo

In [ ]:
# @title 🧹 LIMPIEZA DE DATASET (Audios cortos)
# Filtra audios menores a 0.5s que causan error en VITS

!pip install --quiet onnxruntime
!python -m piper_train.preprocess \
  --input-dir datasets/sig \
  --output-dir datasets/sig \
  --language es \
  --sample-rate 22050 \
  --dataset-format ljspeech \
  --single-speaker

# Configuración de entrenamiento
DATASET_DIR = "datasets/sig"  # ⬅️ AJUSTAR
MAX_EPOCHS = 3000  # Aumentado porque el checkpoint base ya tiene >2000 épocas
BATCH_SIZE = 10  # Reducido para estabilidad en VRAM T4
CHECKPOINT_EPOCHS = 5
VALIDATION_SPLIT = 0.05
NUM_TEST_EXAMPLES = 0
PRECISION = "32"
QUALITY = "high"  # 🔥 CRÍTICO: Debe coincidir con el checkpoint

# Entrenar modelo con piper_train
!python -m piper_train \
  --dataset-dir {DATASET_DIR} \
  --accelerator gpu \
  --devices 1 \
  --batch-size {BATCH_SIZE} \
  --validation-split {VALIDATION_SPLIT} \
  --num-test-examples {NUM_TEST_EXAMPLES} \
  --max_epochs {MAX_EPOCHS} \
  --resume_from_checkpoint models_base/en_US-lessac-high.ckpt \
  --checkpoint-epochs {CHECKPOINT_EPOCHS} \
  --precision {PRECISION} \
  --quality {QUALITY}  # ⬅️ IMPORTANTE


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
piper-train 1.0.0 requires cython<1,>=0.29.0, which is not installed.
piper-train 1.0.0 requires torch<2,>=1.11.0, but you have torch 2.9.1 which is incompatible.
INFO:preprocess:Single speaker dataset
INFO:

### ⚙️ Configuración Avanzada

Ajusta estos parámetros según tu caso:

- **BATCH_SIZE**:
  - 16-32 para T4 (16GB)
  - 8-16 para GPUs con menos VRAM
  - Reduce si obtienes errores de memoria
  
- **MAX_EPOCHS**:
  - Mínimo 100 para resultados aceptables
  - 500-1000+ para mejor calidad
  
- **CHECKPOINT_EPOCHS**: Cada cuántas épocas guardar un checkpoint

## 6️⃣ Monitorear Progreso

In [ ]:
# Monitorear el progreso durante el entrenamiento
import time
from IPython.display import clear_output

def monitor_training(dataset_dir, update_interval=30):
    """
    Monitorea el entrenamiento en tiempo real

    Args:
        dataset_dir: Directorio del dataset
        update_interval: Segundos entre actualizaciones
    """
    import pandas as pd
    from pathlib import Path

    logs_dir = Path(dataset_dir) / "lightning_logs"

    while True:
        try:
            clear_output(wait=True)

            # Buscar última versión
            versions = sorted(logs_dir.glob("version_*"))
            if not versions:
                print("⏳ Esperando inicio del entrenamiento...")
                time.sleep(update_interval)
                continue

            metrics_file = versions[-1] / "metrics.csv"
            if metrics_file.exists():
                df = pd.read_csv(metrics_file)

                print("=" * 60)
                print("📊 PROGRESO DEL ENTRENAMIENTO")
                print("=" * 60)
                print(f"\nÉpoca actual: {int(df['epoch'].max())}")
                print(f"Total de pasos: {int(df['step'].max())}")
                print(f"\n--- Últimas 5 métricas ---")
                print(df[['epoch', 'loss_gen_all', 'loss_disc_all']].tail())
                print("\n💡 Presiona el botón STOP para detener el monitoreo")

            time.sleep(update_interval)

        except KeyboardInterrupt:
            print("\n✅ Monitoreo detenido")
            break
        except Exception as e:
            print(f"⚠️ Error: {e}")
            time.sleep(update_interval)

# Para iniciar el monitoreo (ejecutar en otra celda mientras entrena):
# monitor_training(DATASET_DIR, update_interval=30)

In [ ]:
# Ver métricas
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

logs_dir = Path(DATASET_DIR) / "lightning_logs"
versions = sorted(logs_dir.glob("version_*"))
if versions:
    metrics = versions[-1] / "metrics.csv"
    if metrics.exists():
        df = pd.read_csv(metrics)
        print(df.tail())

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(df['epoch'], df['loss_gen_all'])
        plt.title('Generator Loss')
        plt.subplot(1, 2, 2)
        plt.plot(df['epoch'], df['loss_disc_all'])
        plt.title('Discriminator Loss')
        plt.tight_layout()
        plt.show()

## 7️⃣ Exportar Modelo

In [ ]:
# Exportar a ONNX
from pathlib import Path

# Buscar el último checkpoint
logs_dir = Path(DATASET_DIR) / "lightning_logs"
versions = sorted(logs_dir.glob("version_*"))

if not versions:
    print("❌ No se encontraron versiones de entrenamiento")
else:
    latest_version = versions[-1]
    checkpoints = sorted(latest_version.glob("checkpoints/*.ckpt"))

    if not checkpoints:
        print("❌ No se encontraron checkpoints")
    else:
        CHECKPOINT = str(checkpoints[-1])
        print(f"📦 Exportando checkpoint: {CHECKPOINT}")

        # Crear directorio de salida
        !mkdir -p outputs

        # Exportar a ONNX usando el módulo correcto
        !python3 -m piper_train.export_onnx \
            "{CHECKPOINT}" \
            "/content/piper-training/outputs/model.onnx"

        # Copiar archivo de configuración JSON
        config_src = Path(DATASET_DIR) / "config.json"
        if config_src.exists():
            !cp "{config_src}" outputs/model.onnx.json
            print("✅ Modelo exportado a outputs/")
            print(f"   - model.onnx")
            print(f"   - model.onnx.json")
        else:
            print("⚠️ No se encontró config.json en el dataset")

## 8️⃣ Descargar Resultados

In [ ]:
# Guardar en Google Drive
!mkdir -p "/content/drive/MyDrive/piper-models/trained_model"
!cp -r outputs/* "/content/drive/MyDrive/piper-models/trained_model/"
print("✅ Guardado en Drive")

In [ ]:
# O descargar directamente
from google.colab import files
!zip -r model_trained.zip outputs/
files.download("model_trained.zip")

## 9️⃣ Probar el Modelo

Prueba tu modelo entrenado directamente en Colab

In [ ]:
# Instalar piper-tts para pruebas
!pip install -q piper-tts

# Generar audio de prueba
TEST_TEXT = "Hola, esta es una prueba de mi voz personalizada con Piper TTS."
OUTPUT_WAV = "prueba.wav"

!echo "{TEST_TEXT}" | piper --model outputs/model.onnx --output_file {OUTPUT_WAV}

# Reproducir audio
from IPython.display import Audio, display
import os

if os.path.exists(OUTPUT_WAV):
    print("✅ Audio generado exitosamente")
    display(Audio(OUTPUT_WAV, autoplay=False))
else:
    print("❌ No se pudo generar el audio")

## 🔟 Solución de Problemas

### Errores comunes y soluciones:

**1. Error de memoria GPU (CUDA out of memory)**
- Reduce `BATCH_SIZE` (prueba con 8, 4, o incluso 2)
- Cierra otras celdas que usen GPU
- Reinicia el runtime: `Runtime > Restart runtime`

**2. Error "No module named 'piper_train'"**
- Verifica que ejecutaste la celda de instalación de dependencias
- Asegúrate de estar en el directorio correcto

**3. Dataset vacío o sin muestras**
- Verifica que `dataset.jsonl` tenga contenido
- Revisa que los archivos de audio estén en el formato correcto
- Ejecuta la celda de verificación de dataset

**4. Pérdidas (losses) no disminuyen**
- Es normal al principio del entrenamiento
- Espera al menos 50-100 épocas antes de evaluar
- Verifica que el modelo base sea compatible con tu idioma

**5. Entrenamiento muy lento**
- Verifica que estés usando GPU: `Runtime > Change runtime type > GPU`
- Comprueba disponibilidad con `!nvidia-smi`
- GPU T4 es recomendada para mejor rendimiento

## 🧹 Limpieza (Opcional)

Libera espacio eliminando archivos temporales

In [ ]:
# Eliminar archivos temporales para liberar espacio
import shutil
from pathlib import Path

print("🧹 Limpiando archivos temporales...\n")

# Eliminar logs antiguos (mantener solo el último)
logs_dir = Path(DATASET_DIR) / "lightning_logs"
if logs_dir.exists():
    versions = sorted(logs_dir.glob("version_*"))
    if len(versions) > 1:
        for old_version in versions[:-1]:
            shutil.rmtree(old_version)
            print(f"✅ Eliminado: {old_version.name}")

# Eliminar checkpoints intermedios (mantener solo cada 10 épocas)
if versions:
    checkpoints_dir = versions[-1] / "checkpoints"
    if checkpoints_dir.exists():
        checkpoints = sorted(checkpoints_dir.glob("*.ckpt"))
        for ckpt in checkpoints[:-1]:  # Mantener el último
            epoch = ckpt.stem.split('-')[0].replace('epoch=', '')
            if epoch.isdigit() and int(epoch) % 10 != 0:
                ckpt.unlink()
                print(f"✅ Eliminado checkpoint: {ckpt.name}")

# Verificar espacio disponible
total, used, free = shutil.disk_usage("/")
print(f"\n💾 Espacio disponible: {free // (2**30)} GB")
print("✅ Limpieza completada")